# Java

Java is the mother of all JVM languages, first released in 1995 after years of development by Sun Microsystems.  BeakerX uses [OpenJDK](http://openjdk.java.net/) for Java and all the kernels.

BeakerX supports Java cells that define classes, which can be used by other JVM based cells.
One cell is equivalent to a Java compilation unit and can contain a single class, or a sequence of statements.

In [1]:
%classpath add mvn org.json json 20180813

In [13]:
import org.json.JSONObject;
import org.json.JSONArray;

import java.io.FileReader;
import java.io.BufferedReader;

import java.io.FileNotFoundException;
import java.io.IOException;

import java.io.InputStreamReader;
import java.net.URL;
import java.net.URLEncoder;
import java.util.Collection;
import java.util.List;
import java.util.ArrayList;
import java.util.Scanner;


        URL url= new URL("https://api-adresse.data.gouv.fr/search/?q="+URLEncoder.encode("gare de Lyon", "UTF-8")+"&postcode=75000");
        try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
            JSONArray res = new JSONObject(br.readLine()).getJSONArray("features").getJSONObject(0).getJSONObject("geometry").getJSONArray("coordinates");
    System.err.println(res);
        }


[2.3469,48.8589]


null

In [ ]:
package test.beaker;

import java.util.Date;
import java.text.SimpleDateFormat;

public class BeakerTest {
  private Date _date;
  private SimpleDateFormat sdf = new SimpleDateFormat("yyyy-MM-dd'T'HH:mmZ");
  
  public BeakerTest() {
    _date = new Date();
  }

  public String getDateTxt() {
    return "Today:" + sdf.format(_date);
  }

  public String getDateUpperCaseTxt() {
    return getDateTxt().toUpperCase();
  }

}


In [ ]:
import java.io.FileReader;
import java.io.BufferedReader;

import java.io.FileNotFoundException;
import java.io.IOException;

import java.io.InputStreamReader;
import java.net.URL;
import java.net.URLEncoder;

import org.json.JSONObject;
import org.json.JSONArray;

import java.util.Collection;
import java.util.List;
import java.util.ArrayList;
import java.util.Scanner;

public class GeoPoint{
    private final double latitude;
    private final double longitude;
    private final List<GeoPoint> neighbors;
    public GeoPoint(double latitude, double longitude){
        this.latitude= latitude;
        this.longitude= longitude;
	neighbors= new ArrayList<GeoPoint>();
    }

    public double distanceTo(GeoPoint other){
        return distanceTo(other.latitude, other.longitude);
    }
   public double distanceTo(double latitude, double longitude){
       double earthRadius= 6371e3;// in meters
        double phi1=Math.toRadians(this.latitude);
        double phi2=Math.toRadians(latitude);
        double deltaPhi=Math.toRadians(latitude- this.latitude);
        double deltaLambda= Math.toRadians(longitude- this.longitude);
        double a= Math.sin(deltaPhi/2) * Math.sin(deltaPhi/2) +
            Math.cos(phi1) * Math.cos(phi2) * Math.sin(deltaLambda/2) * Math.sin(deltaLambda/2);
        double c= 2* Math.atan2(Math.sqrt(a), Math.sqrt(1-a));
        return earthRadius * c;
    }
    public String toString(){
        return "GeoPoint: "+super.toString()+"{latitude: "+latitude+", longitude: " + longitude+", nb of neighbors"+neighbors.size()+"}";
    }
    public static GeoPoint closerTo(double latitude, double longitude, double deltaD, Collection<GeoPoint> points){
        GeoPoint res=null;
        double minD= Double.POSITIVE_INFINITY;
        for(GeoPoint point : points){
            double currentD= point.distanceTo(latitude, longitude);
            if((currentD < minD) && (currentD > deltaD) ){
                minD= currentD;
                res= point;
            }
        }
        return res;
    }
    public static GeoPoint findByName(String name){
	    GeoPoint res= null;
        URL url= new URL("https://api-adresse.data.gouv.fr/search/?q="+URLEncoder.encode(name, "UTF-8")+"&postcode=75000");
        try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
            JSONArray coords = new JSONObject(br.readLine()).getJSONArray("features").getJSONObject(0).getJSONObject("geometry").getJSONArray("coordinates");
            res= new GeoPoint(coords.getDouble(0), coords.getDouble(1));
        }catch(Exception e){
            System.out.println(e);
        }
        return res; // should throw
        }

    public static GeoPoint closerTo(String name, Collection<GeoPoint> data){
        return closerTo(findByName(name), data);
    }
    public static GeoPoint closerTo(GeoPoint ref, Collection<GeoPoint> data){
        return closerTo(ref.latitude, ref.longitude, 1.e-50, data);
    }
    public static List<Ville> read(String villesURL) throws IOException {
        List<Ville> res= new ArrayList<Ville>();
        URL url= new URL(villesURL);
        try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
            br.readLine(); // skip header
            for(String line = br.readLine(); line != null; line= br.readLine()){
                String[] data = line.split(";");
                //System.err.println(line+":"+data.length);
                if(data.length == 3){
                    for(int i=1; i !=3; ++i){
                        data[i]= data[i].replace(',','.');
                    }
                    try{
                        res.add(new Ville(data[0], Double.parseDouble(data[1]), Double.parseDouble(data[2])));
                    }catch(NumberFormatException e){
                        System.err.println(e);
                    }
                }
            }
        }
        return res;
    }
    public static void main(String[] args){
        try{
            String url="https://gist.githubusercontent.com/bhugueney/e48ad6459a98715a6db69f59f1ba992f/raw/e06c8ad6bd10e8462aaa9ca6753c1266b8478afc/NameLatLong.txt";
            List<Ville> villes= read(url);
            System.out.println(closerTo(48.76,1.36, 1.e-29, villes));
            System.out.println(closerTo("Dreux", villes));

        }catch(FileNotFoundException e){
            System.err.println(e);
        }
        catch(IOException e){
            System.err.println(e);
        }
    }
    
}


In [ ]:
package test.beaker;

BeakerTest bt = new BeakerTest();
return bt.getDateTxt();

In [ ]:
import java.util.List;
import java.util.ArrayList;
import com.twosigma.beakerx.chart.xychart.Plot;
import com.twosigma.beakerx.chart.xychart.plotitem.*;
import com.twosigma.beakerx.chart.Color;

Plot p = new Plot();

p.setTitle("this is a Java plot");

Bars b = new Bars();

List<Number> yList = new ArrayList<Number>();
yList.add(2);
yList.add(5);
yList.add(4);
yList.add(8);

b.setY(yList);
b.setColor(Color.blue);
b.setWidth(0.5);

p.add(b);
  
return p;

In [ ]:
package test.beaker;
interface DateGetter {
   public String getDateTxt();
}

In [ ]:
package test.beaker;
public class DG2 extends BeakerTest implements DateGetter {
}

## Using the classpath to load a jar

In [ ]:
%classpath add jar ../resources/jar/BeakerXClasspathTest.jar

In [ ]:
System.out.println("The Groovy working folder is :");
System.out.println(java.nio.file.Paths.get(".").toAbsolutePath().normalize().toString());
System.out.println("BeakerXClasspathTest.jar exists in this folder");

In [ ]:
import com.beaker.BeakerXClasspathTest;

BeakerXClasspathTest t = new BeakerXClasspathTest();
System.out.println(com.beaker.BeakerXClasspathTest.staticTest);
System.out.println(t.getObjectTest());